In [ ]:
# Imports
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv

import os
import sys
import random
import importlib
import datetime
import warnings
import math
import cmath
import pycircstat as circ
warnings.filterwarnings('ignore')

from pprint import pprint
from itertools import zip_longest
from scipy.stats import sem, norm, binned_statistic, percentileofscore
from scipy.optimize import least_squares

from bokeh.resources import INLINE
from bokeh.io import export_svgs, export_png
from holoviews import opts, dim
from holoviews.operation import histogram
hv.extension('bokeh')
# hv.extension('matplotlib')
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import paths
import processing_parameters
import functions_bondjango as bd
import functions_plotting as fp
import functions_data_handling as fdh
import functions_kinematic as fk

importlib.reload(fp)
# set up the figure theme
fp.set_theme()

# Functions

## Utilities

In [ ]:
def list_lists_to_array(list_of_lists):
    """ Converts a list of lists into a 2D array

    Parameters
    ----------
    list_of_lists : list

    Returns
    -------
    new_array : np.array
        Array where each row was an entry in the list of lists
    """

    max_length = max([len(sublist) for sublist in list_of_lists])
    new_array = np.empty((len(list_of_lists), max_length))
    new_array[:] = np.NaN

    for row, l in enumerate(list_of_lists):
        new_array[row, :len(l)] = l

    return new_array

In [ ]:
def normalize_rows(data_in):
    
    for idx, el in enumerate(data_in):
        data_in[idx, :] = (el-np.nanmin(el))/(np.nanmax(el)-np.nanmin(el))
    return data_in

# Load the Data

In [ ]:
importlib.reload(processing_parameters)
# get the search string
search_string = processing_parameters.search_string


# get the paths from the database
file_path, paths_all, parsed_query, date_list, animal_list = fdh.fetch_preprocessing(processing_parameters.search_string)

animal_idxs = [i for i,d in enumerate(animal_list) if d==parsed_query['mouse'].lower()]
good_entries = [file_path[index] for index in animal_idxs]
input_path = [paths_all[index] for index in animal_idxs]

# # assemble the output path
# out_path = os.path.join(paths.analysis_path, 'test_latentconsolidate.hdf5')
print(input_path)

In [ ]:
# allocate memory for the data
free_data = []
fixed_data = []
data = []
exp_type = []

for files in input_path:
    print(files)
    # load the data
    with pd.HDFStore(files) as h:
#         beh_data.append(h['full_traces'])
        if '/matched_calcium' in h.keys():
            
            data.append(h['matched_calcium'])
            
            if "VWheel" in files:
                fixed_data.append(h['matched_calcium'])
                exp_type.append('fixed')
            else:
                free_data.append(h['matched_calcium'])
                exp_type.append('free')

                
print(f'Number of files loaded: {len(data)}')

# data = {'free': pd.concat(free_data), 'fixed': pd.concat(fixed_data)}

# Calculate orientation for each dataset
for ds in data:
# for name, ds in data.items():
    ds['orientation'] = ds['direction']
    ds['orientation'][ds['orientation'] < 0] += 180
    
# Grab list of direction and orientations used
directions = np.sort(ds.direction.unique()[1:])
directions_wrapped = np.sort(fk.wrap(directions))
orientations = np.sort(ds.orientation.unique()[1:])
pprint(f'Directions used: {directions}')
pprint(f'Orientations used: {orientations}')

# Remove baselines from the data for later processing

In [ ]:
keys = ['direction', 'orientation']

# Allocate memory for cell property dataframes
df_list = []

for ds in data:

    # get the number of cells
    cells = [el for el in ds.columns if 'cell' in el]
    
    # allocate memory for the experiment
    exp_list = []

    # Get the 7th percentile of activity per cell for each stimulus
    # Try 7th/8th percentile
    percentiles = ds.groupby(['direction'])[cells].quantile(0.07)

    # get the baselines - The first column is the inter-trial interval
    baselines = percentiles.iloc[0, percentiles.columns.get_loc(cells[0]):percentiles.columns.get_loc(cells[-1])+1]

    # Subtract baseline from everything
    ds[cells].subtract(baselines, axis=1)

# Freely Moving

## Running traces and distributions
Seems like 50th percentile is a decent cutoff for running vs not running periods

In [ ]:
# Make time trace of data
trace = hv.Curve((data[0].time_vector, data[0].mouse_speed)).opts(xlabel='time', ylabel='m/s', width=600, height=300)
percenitile = np.percentile(np.abs(data[0].mouse_speed), 50)
percentile_line = hv.HLine(percenitile).opts(color='r', line_width=2)

# Make histogram
# Create time bins of 0.5sec length with mean running speed
time_bins = np.arange(0, data[0].time_vector.max(), step=0.25)
vals, _, _ = binned_statistic(data[0].time_vector, data[0].mouse_speed, bins=time_bins, statistic=np.nanmean)

hist_bins = np.arange(0, np.max(vals), step=0.005)
frequencies, edges = np.histogram(vals, hist_bins)
print('Values: %s, Edges: %s' % (frequencies.shape[0], edges.shape[0]))
hist = hv.Histogram((edges, frequencies)).opts(xlabel='m/s', width=600,)
percentile_line2 = hv.VLine(percenitile).opts(color='r', line_width=2)

(trace*percentile_line + hist*percentile_line2).opts(shared_axes=False)

## Angular speed trace and distribution

In [ ]:
# Make time trace of data
trace = hv.Curve((data[0].time_vector, np.abs(data[0].mouse_angular_speed))).opts(xlabel='time', ylabel='m/s', width=600, height=300)
percenitile = np.percentile(np.abs(data[0].mouse_angular_speed), 50)
percentile_line = hv.HLine(percenitile).opts(color='r', line_width=2)

# Make histogram
# Create time bins of 0.5sec length with mean running speed
time_bins = np.arange(0, data[0].time_vector.max(), step=0.25)
vals, _, _ = binned_statistic(data[0].time_vector, np.abs(data[0].mouse_angular_speed), bins=time_bins, statistic=np.nanmean)

hist_bins = np.arange(0, np.max(vals), step=10)
frequencies, edges = np.histogram(vals, hist_bins)
print('Values: %s, Edges: %s' % (frequencies.shape[0], edges.shape[0]))
hist = hv.Histogram((edges, frequencies)).opts(xlabel='m/s', width=600,)
percentile_line2 = hv.VLine(percenitile).opts(color='r', line_width=2)

(trace*percentile_line + hist*percentile_line2).opts(shared_axes=False)

## Head angle traces
Note here that y is the vertical axis (yaw), x is forward (roll), and z is side (pitch)

In [ ]:
overlay = hv.Curve((data[0].time_vector, data[0].mouse_yrot_m), label='y_rot') * \
            hv.Curve((data[0].time_vector, data[0].mouse_zrot_m+360), label='z_rot') * \
            hv.Curve((data[0].time_vector, data[0].mouse_xrot_m+720), label='x_rot')
overlay.opts(xlabel='time', width=900, height=600)

In [ ]:
# Remove jumps from traces - doesn't fix 0-360 jumps, but it doesn't need to
rot_traces = ['mouse_xrot_m', 'mouse_yrot_m', 'mouse_zrot_m']
data[0][rot_traces] = data[0][rot_traces].apply(fk.jump_killer, args=[50])
data[0][rot_traces] = data[0][rot_traces].apply(fk.wrap)

overlay = hv.Curve((data[0].time_vector, data[0].mouse_yrot_m), label='y_rot') * \
            hv.Curve((data[0].time_vector, data[0].mouse_zrot_m+360), label='z_rot') * \
            hv.Curve((data[0].time_vector, data[0].mouse_xrot_m+720), label='x_rot')
overlay.opts(xlabel='time', width=900, height=600)

# Head fixed
Seems like 80th percentile is a decent cutoff for running vs not running periods

In [ ]:
# Get time trace
hv.Curve((data[1].time_vector, np.abs(data[1].wheel_speed))).opts(xlabel='time', ylabel='m/s', width=600, height=300)
percenitile = np.percentile(np.abs(data[1].wheel_speed), 80)
percentile_line = hv.HLine(percenitile).opts(color='r', line_width=2)


# Make histogram
# Create time bins of 0.5sec length with mean running speed
time_bins = np.arange(0, data[1].time_vector.max(), step=0.25)
vals, _, _ = binned_statistic(data[1].time_vector, np.abs(data[1].wheel_speed), bins=time_bins, statistic=np.nanmean)

hist_bins = np.arange(0, np.max(vals), step=0.005)
frequencies, edges = np.histogram(vals, hist_bins)
print('Values: %s, Edges: %s' % (frequencies.shape[0], edges.shape[0]))
hist = hv.Histogram((edges, frequencies)).opts(xlabel='m/s', width=600,)
percentile_line2 = hv.VLine(percenitile).opts(color='r', line_width=2)

(trace*percentile_line + hist*percentile_line2).opts(shared_axes=False)